# Projeto: *Music Generation*
* Geração da próxima nota musical.

Faça o [download](https://drive.google.com/drive/folders/1dKOdOhmcijZhoZEYwDJ-to3O4jAfiiD_?usp=sharing) de um conjunto de dados dos Corais de Bach.

Este conjunto é composto por 382 corais compostos por Johann Sebastian Bach. 

Cada coral tem entre 100 e 640 passos de tempo e cada passo de tempo contém 4 inteiros, onde cada inteiro corresponde ao índice de uma nota em um piano (exceto pelo valor 0, que significa que nenhuma nota é tocada). 

Treine um modelo - recorrente, convolucional ou ambos - que possa prever o próximo passo de tempo (quatro notas), dado uma sequência de passos de tempo de um coral. 

Em seguida, use esse modelo para gerar música semelhante a Bach, uma nota de cada vez: você pode fazer isso fornecendo ao modelo o início de um coral e pedindo para ele prever o próximo passo de tempo, em seguida, acrescentando esses passos de tempo à sequência de entrada e pedindo ao modelo pela próxima nota, e assim por diante. 

## Setup

In [ ]:
# Python ≥3.5 - requerido
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20
import sklearn
assert sklearn.__version__ >= "0.20"

# TensorFlow ≥2.0
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

# Imports comuns
import numpy as np
import os
from pathlib import Path

# Estabilizar e padronizar as respostas deste notebook.
np.random.seed(42)
tf.random.set_seed(42)

# Figuras
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Salvando as figuras
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "rnn"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

## Dataset contendo os corais de Bach

In [ ]:
# Acessando e baixando os arquivos
DOWNLOAD_ROOT = "https://tinyurl.com/2d2ammnn/"
FILENAME = "jsb_chorales.tgz"
filepath = keras.utils.get_file(FILENAME,
                                DOWNLOAD_ROOT + FILENAME,
                                cache_subdir="datasets/jsb_chorales",
                                extract=True)

117661/117661 [==============================] - 0s 0us/step


In [ ]:
# Obtém o diretório pai do caminho de arquivo especificado
jsb_chorales_dir = Path(filepath).parent

# Encontra todos os arquivos que correspondem ao padrão "train/chorale_*.csv" no diretório
train_files = sorted(jsb_chorales_dir.glob("train/chorale_*.csv"))

# Encontra todos os arquivos que correspondem ao padrão "valid/chorale_*.csv" no diretório
valid_files = sorted(jsb_chorales_dir.glob("valid/chorale_*.csv"))

# Encontra todos os arquivos que correspondem ao padrão "test/chorale_*.csv" no diretório
test_files = sorted(jsb_chorales_dir.glob("test/chorale_*.csv"))


In [ ]:
import pandas as pd

def load_chorales(filepaths):
    # Carrega cada arquivo CSV em filepaths usando o pandas e converte o conteúdo para uma lista de listas
    return [pd.read_csv(filepath).values.tolist() for filepath in filepaths]

# Carrega os dados dos arquivos de treinamento usando a função load_chorales
train_chorales = load_chorales(train_files)

# Carrega os dados dos arquivos de validação usando a função load_chorales
valid_chorales = load_chorales(valid_files)

# Carrega os dados dos arquivos de teste usando a função load_chorales
test_chorales = load_chorales(test_files)


In [ ]:
train_chorales[0]

[[74, 70, 65, 58],
 [74, 70, 65, 58],
 [74, 70, 65, 58],
 [74, 70, 65, 58],
 [75, 70, 58, 55],
 [75, 70, 58, 55],
 [75, 70, 60, 55],
 [75, 70, 60, 55],
 [77, 69, 62, 50],
 [77, 69, 62, 50],
 [77, 69, 62, 50],
 [77, 69, 62, 50],
 [77, 70, 62, 55],
 [77, 70, 62, 55],
 [77, 69, 62, 55],
 [77, 69, 62, 55],
 [75, 67, 63, 48],
 [75, 67, 63, 48],
 [75, 69, 63, 48],
 [75, 69, 63, 48],
 [74, 70, 65, 46],
 [74, 70, 65, 46],
 [74, 70, 65, 46],
 [74, 70, 65, 46],
 [72, 69, 65, 53],
 [72, 69, 65, 53],
 [72, 69, 65, 53],
 [72, 69, 65, 53],
 [72, 69, 65, 53],
 [72, 69, 65, 53],
 [72, 69, 65, 53],
 [72, 69, 65, 53],
 [74, 70, 65, 46],
 [74, 70, 65, 46],
 [74, 70, 65, 46],
 [74, 70, 65, 46],
 [75, 69, 63, 48],
 [75, 69, 63, 48],
 [75, 67, 63, 48],
 [75, 67, 63, 48],
 [77, 65, 62, 50],
 [77, 65, 62, 50],
 [77, 65, 60, 50],
 [77, 65, 60, 50],
 [74, 67, 58, 55],
 [74, 67, 58, 55],
 [74, 67, 58, 53],
 [74, 67, 58, 53],
 [72, 67, 58, 51],
 [72, 67, 58, 51],
 [72, 67, 58, 51],
 [72, 67, 58, 51],
 [72, 65, 57

Notas de 36 (C1 = C na oitava 1) até 81 (A5 = A na oitava 5) e 0 para
silêncio.

Por exemplo: `[74, 70, 65, 58]` corresponde à: `[D, Bb, F, B,]` - Acorde de Bb

In [ ]:
# Criando os acordes
notes = set()  # Cria um conjunto vazio para armazenar as notas

# Percorre os conjuntos de treinamento, validação e teste
for chorales in (train_chorales, valid_chorales, test_chorales):
    for chorale in chorales:  # Percorre cada chorale dentro do conjunto
        for chord in chorale:  # Percorre cada chord dentro do chorale
            notes |= set(chord)  # Adiciona as notas do chord ao conjunto 'notes'

n_notes = len(notes)  # Obtém o número total de notas únicas
min_note = min(notes - {0})  # Encontra a nota mínima, excluindo a nota 0
max_note = max(notes)  # Encontra a nota máxima

# Verifica se as notas mínima e máxima estão corretas
assert min_note == 36
assert max_note == 81


Neste código, um conjunto vazio chamado `notes` é criado para armazenar as notas encontradas nos dados musicais. Em seguida, um loop é realizado sobre os conjuntos de treinamento, validação e teste. Dentro de cada conjunto, são percorridos os chorales e, em seguida, os chords de cada chorale. As notas de cada chord são adicionadas ao conjunto `notes` usando a operação de união (`|=`).

Após o loop, o número total de notas únicas é obtido através do comprimento do conjunto `notes` armazenado na variável `n_notes`. A nota mínima é encontrada usando a função `min`, excluindo a nota 0 do conjunto `notes - {0}` e armazenada na variável `min_note`. A nota máxima é encontrada usando a função `max` no conjunto `notes` e armazenada na variável `max_note`.

Em seguida, o código verifica se as notas mínima e máxima estão corretas usando as declarações assert. Se as verificações falharem, será lançada uma exceção para indicar que algo está errado. Neste caso, o código verifica se `min_note` é igual a 36 e se `max_note` é igual a 81.

## Ouvindo o Dataset

Vamos escrever algumas funções para ouvir esses chorais escrevendo um sintetizador para traduzir o dataset em audio.

In [ ]:
from IPython.display import Audio

def notes_to_frequencies(notes):
    # A frequência dobra ao subir uma oitava; existem 12 semitons em cada oitava;
    # A nota A na oitava 4 é 440 Hz, e ela corresponde ao número 69.
    return 2 ** ((np.array(notes) - 69) / 12) * 440

def frequencies_to_samples(frequencies, tempo, sample_rate):
    note_duration = 60 / tempo  # o tempo é medido em batidas por minuto
    # Para reduzir o som de clique a cada batida, arredondamos as frequências para tentar
    # obter as amostras próximas de zero no final de cada nota.
    frequencies = np.round(note_duration * frequencies) / note_duration
    n_samples = int(note_duration * sample_rate)
    time = np.linspace(0, note_duration, n_samples)
    sine_waves = np.sin(2 * np.pi * frequencies.reshape(-1, 1) * time)
    # Removendo todas as notas com frequências ≤ 9 Hz (inclui a nota 0 = silêncio)
    sine_waves *= (frequencies > 9.).reshape(-1, 1)
    return sine_waves.reshape(-1)

def chords_to_samples(chords, tempo, sample_rate):
    freqs = notes_to_frequencies(chords)
    freqs = np.r_[freqs, freqs[-1:]]  # faz a última nota um pouco mais longa
    merged = np.mean([frequencies_to_samples(melody, tempo, sample_rate)
                     for melody in freqs.T], axis=0)
    n_fade_out_samples = sample_rate * 60 // tempo  # fade out na última nota
    fade_out = np.linspace(1., 0., n_fade_out_samples)**2
    merged[-n_fade_out_samples:] *= fade_out
    return merged

def play_chords(chords, tempo=160, amplitude=0.1, sample_rate=44100, filepath=None):
    samples = amplitude * chords_to_samples(chords, tempo, sample_rate)
    if filepath:
        from scipy.io import wavfile
        samples = (2**15 * samples).astype(np.int16)
        wavfile.write(filepath, sample_rate, samples)
        return display(Audio(filepath))
    else:
        return display(Audio(samples, rate=sample_rate))


In [7]:
for index in range(3):
    play_chords(train_chorales[index])